# Supply Chain Analytics Dashboard

This notebook analyzes logistics KPIs using synthetic shipment data. We explore cost trends, warehouse throughput, customs clearance performance, and on-time delivery rates across carriers and regions.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

## Generate Synthetic Logistics Data

In [ ]:
np.random.seed(42)

n_shipments = 1000
warehouses = ['Warehouse_A', 'Warehouse_B', 'Warehouse_C', 'Warehouse_D', 'Warehouse_E']
regions = ['North', 'South', 'East', 'West']
carriers = ['FedEx', 'UPS', 'DHL', 'USPS']

dates = pd.date_range(start='2024-01-01', end='2024-12-31', periods=n_shipments)
dates = np.random.choice(dates, size=n_shipments, replace=True)

df = pd.DataFrame({
    'shipment_id': [f'SHP{i:05d}' for i in range(1, n_shipments + 1)],
    'date': dates,
    'origin_warehouse': np.random.choice(warehouses, size=n_shipments),
    'destination_region': np.random.choice(regions, size=n_shipments),
    'weight_kg': np.random.uniform(1, 500, size=n_shipments).round(2),
    'shipping_cost': np.random.uniform(15, 350, size=n_shipments).round(2),
    'delivery_days': np.random.randint(1, 14, size=n_shipments),
    'carrier': np.random.choice(carriers, size=n_shipments),
    'customs_clearance_hours': np.random.exponential(12, size=n_shipments).round(2),
    'on_time_delivery': np.random.choice([True, False], size=n_shipments, p=[0.78, 0.22])
})

df['date'] = pd.to_datetime(df['date'])

print('Data Preview:')
display(df.head())
print('\nDescriptive Statistics:')
display(df.describe())

## Cost Per Shipment Trends

In [ ]:
df['month'] = df['date'].dt.to_period('M').astype(str)
monthly_cost = df.groupby(['month', 'carrier'])['shipping_cost'].mean().reset_index()

fig = px.line(monthly_cost, x='month', y='shipping_cost', color='carrier',
              title='Monthly Average Cost Per Shipment by Carrier',
              labels={'shipping_cost': 'Average Shipping Cost ($)', 'month': 'Month'})
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## Warehouse Throughput Analysis

In [ ]:
warehouse_stats = df.groupby('origin_warehouse').agg(
    total_shipments=('shipment_id', 'count'),
    avg_weight_kg=('weight_kg', 'mean')
).reset_index()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Total Shipments by Warehouse', 'Average Weight by Warehouse'))

fig.add_trace(
    go.Bar(x=warehouse_stats['origin_warehouse'], y=warehouse_stats['total_shipments'], name='Shipments'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=warehouse_stats['origin_warehouse'], y=warehouse_stats['avg_weight_kg'], name='Avg Weight (kg)'),
    row=1, col=2
)

fig.update_layout(title_text='Warehouse Throughput Analysis', showlegend=False)
fig.update_xaxes(title_text='Origin Warehouse', row=1, col=1)
fig.update_xaxes(title_text='Origin Warehouse', row=1, col=2)
fig.update_yaxes(title_text='Total Shipments', row=1, col=1)
fig.update_yaxes(title_text='Average Weight (kg)', row=1, col=2)
fig.show()

## Customs Clearance Performance

In [ ]:
fig = px.box(df, x='destination_region', y='customs_clearance_hours',
             title='Customs Clearance Hours by Destination Region',
             labels={'customs_clearance_hours': 'Clearance Hours', 'destination_region': 'Destination Region'})
fig.show()

## On-Time Delivery Rate by Carrier

In [ ]:
on_time_rate = df.groupby('carrier')['on_time_delivery'].mean().reset_index()
on_time_rate['on_time_pct'] = (on_time_rate['on_time_delivery'] * 100).round(1)

fig = px.bar(on_time_rate, x='carrier', y='on_time_pct', color='on_time_pct',
             color_continuous_scale=['#d62728', '#ff7f0e', '#2ca02c'],
             title='On-Time Delivery Rate by Carrier (%)',
             labels={'on_time_pct': 'On-Time Rate (%)', 'carrier': 'Carrier'})
fig.update_layout(showlegend=False, coloraxis_showscale=False)
fig.show()

## Key Findings

- **Cost Trends**: Shipping costs vary by carrier and month; seasonal patterns may emerge in the line chart.
- **Warehouse Throughput**: Some warehouses handle significantly more shipments; average weight distribution differs across locations.
- **Customs Clearance**: Destination region impacts clearance times; certain regions show higher variance in processing hours.
- **On-Time Delivery**: Carrier performance varies; color-coded bars highlight top and bottom performers.